In [ ]:
!pip install easyocr

In [ ]:
'''
!apt-get update -qq
!apt-get install fonts-nanum* -qq
'''

'\n!apt-get update -qq\n!apt-get install fonts-nanum* -qq\n'

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
PATH_PAPER = '../gdrive/MyDrive/공모전용/HW-OCR/4.Validaiton/paper/R.Free/'
PATH_TABLET = '../gdrive/MyDrive/공모전용/HW-OCR/4.Validaiton/tablet/R.Free/'

Mounted at /gdrive


In [ ]:
import easyocr
import os
import PIL
import numpy as np
import cv2
import random
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import ImageFont, ImageDraw, Image
import matplotlib.font_manager as fm  # 폰트 관련 용도

In [ ]:
'''
sys_font=fm.findSystemFonts()
print(f"sys_font number: {len(sys_font)}")
print(sys_font)

nanum_font = [f for f in sys_font if 'Nanum' in f]
print(f"nanum_font number: {len(nanum_font)}")
'''

'\nsys_font=fm.findSystemFonts()\nprint(f"sys_font number: {len(sys_font)}")\nprint(sys_font)\n\nnanum_font = [f for f in sys_font if \'Nanum\' in f]\nprint(f"nanum_font number: {len(nanum_font)}")\n'

In [ ]:
'''
path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자

font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)
fm._rebuild()
'''

"\npath = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자\n\nfont_name = fm.FontProperties(fname=path, size=10).get_name()\nprint(font_name)\nplt.rc('font', family=font_name)\nfm._rebuild()\n"

In [ ]:
def image_loca(reader):
  loca = []

  for red in reader:
    
    prob = red[2]
    if prob < 0.5:
      continue
    
    temp = red[0]
    temp.append(red[1])
    loca.append(temp)

  return loca

In [ ]:
## path = 파일명
def json_data(path):
    f = open(path,encoding = "UTF-8")
    data = json.loads(f.read())

    whole_loca = []
    for i in range(len(data['bbox'])):
        loca = []
        for a, b in zip(data['bbox'][i]['x'],data['bbox'][i]['y']):
            loca.append([a,b])
        loca.append(data['bbox'][i]['data'])
        whole_loca.append(loca)
    
    return whole_loca

In [ ]:
## 공통인자 추출
## 오로지 tablet 폴더 손글씨 용입니다. 공문서는 따로 구성해야 함
def comparing_img_json(img, js):
  img_list = []
  
  for i in img:
    img_list.append(i[-1])
  
  return_img_list = []

  for i in js:
    if i[-1] in img_list:
      return_img_list.append(i)

  return return_img_list

In [ ]:
## 원근변환용 좌표 추출 함수

def cal_center(arr):
    mins = np.min(arr, axis=0)
    maxs = np.max(arr, axis=0)
    
    X_low = mins[0]
    X_max = maxs[0]
    Y_low = mins[1]
    Y_max = maxs[1]
    
    L_H = [X_low, Y_low]
    L_L = [X_low, Y_max]
    R_H = [X_max, Y_low]
    R_L = [X_max, Y_max]
    
    result = np.float32([L_H, L_L, R_H, R_L])
    
    return result

In [ ]:
## 원근 변환 함수
def warp_Perspective(img, js):
    rows, cols = img.shape[:2]
    
    #---① 원근 변환 전 후 4개 좌표
    np_js = np.array(js).reshape(4,2)
    pts1 = cal_center(np_js)

    rows = int(pts1[1][1] - pts1[0][1])
    cols = int(pts1[2][0] - pts1[1][0])
    
    pts2 = np.float32([[0,0], [0,rows-1], [cols-1, 0], [cols-1,rows-1]])

    #---③ 원근 변환 행렬 계산
    mtrx = cv2.getPerspectiveTransform(pts1, pts2)
    #---④ 원근 변환 적용
    dst = cv2.warpPerspective(img, mtrx, (cols, rows))
  

    return dst

In [ ]:
def img_slicing(path, img, word):
  pil_img = Image.fromarray(img)
  word_size = len(word)
  img_size = pil_img.size

  y_slice = img_size[1]
  x_slice = int(img_size[0]/word_size)


  for i in range(word_size):
    croppedImage = pil_img.crop((i*x_slice, 0, (i+1)*x_slice, y_slice))
    save_image(path, word[i], croppedImage)

In [ ]:
def save_image(path, one_word, cropimage):
  directory = path+'/전처리/'+one_word
  global FILENUMBER
  try:
    if not os.path.exists(directory):
      os.makedirs(directory)
  except:
    pass

  filename = directory+'/{}_{}.png'.format(FILENUMBER, one_word)
  if os.path.isfile(filename):
    FILENUMBER+=1
    return
  
  cropimage.save(filename)
  FILENUMBER+=1

In [ ]:
reader = easyocr.Reader(['ko', 'en'])

In [ ]:
filelist = os.listdir(PATH_PAPER)

In [ ]:
filelist.sort()

In [ ]:
filelist[:5]

['IMG_OCR_53_4PR_13001.json',
 'IMG_OCR_53_4PR_13001.png',
 'IMG_OCR_53_4PR_13002.json',
 'IMG_OCR_53_4PR_13002.png',
 'IMG_OCR_53_4PR_13003.json']

In [ ]:
## 파일명 분할 추출
pnglist = []
jsonlist = []
global FILENUMBER
FILENUMBER = 264428
for i in range(len(filelist)):

  if i == len(filelist)-2:
    break

  split1 = filelist[i].split('.')[0]
  split2 = filelist[i+1].split('.')[0]

  if split1 == split2:
    pnglist.append(split1+'.png')
    jsonlist.append(split1+'.json')
  else:
    continue

In [ ]:
len(jsonlist)

3012

In [ ]:
res = []
count = 0
for i, j in tqdm(zip(pnglist, jsonlist)):
  result = reader.readtext(PATH_PAPER+i)
  img = cv2.imread(PATH_PAPER+i)

  img_data = image_loca(result)
  js_data = json_data(PATH_PAPER+j)

  target = comparing_img_json(img_data, js_data)

  for t in target:
    order = t[:-1]
    pre_img = warp_Perspective(img, order)
    try:
      img_slicing(PATH_TABLET, pre_img, t[-1])
    except ValueError:
      print(t, '여기서 불량입니다.')
      pass
  res.append(FILENUMBER)
  if len(res) % 100 == 0:
    print(FILENUMBER)

100it [06:13,  1.96s/it]

270196


200it [09:27,  1.93s/it]

275965


300it [12:32,  1.93s/it]

...

2900it [1:23:37,  1.02it/s]

359050


3000it [1:25:22,  1.10s/it]

360284


3012it [1:25:34,  1.70s/it]
